**TO-DO**
* assinalar que owners é que têm um disponent

## Table Of Contents <a id="index"></a>


- [Dataset managing](#dataset)  
- [District Data](#district-data)
- [Data Exploration](#data-exploration)
- [Matrix](#matrix)
  

#### Models
- [**Decision Tree**](#decision-tree)
    - [**Parameter Tunning**](#parameter-tunning)
- [**K-Nearest Neighbor**](#k-nearest-neighbor)
    - [**Parameter Tunning**](#parameter-tunning-2)  
- [**Support-Vector Machines**](#support-vector-machines)
    - [**Parameter Tunning**](#parameter-tunning-3)
- [**Neural Networks**](#neural-networks)
    - [**Parameter Tunning**](#parameter-tunning-4)
- [**Logistic Regression**](#logistic-regression)
    - [**Parameter Tunning**](#parameter-tunning-5)
- [**Naive Bayes**](#naive-bayes)
    - [**Parameter Tunning**](#parameter-tunning-6)
- [**Random Forest**](#random-forest)
    - [**Parameter Tunning**](#parameter-tunning-7)
- [**XGBoost**](#xgboost)
    - [**Parameter Tunning**](#parameter-tunning-8)
- [**MLP**](#mlp)
    - [**Parameter Tunning**](#parameter-tunning-9)
- [**Ada Boost**](#ada-boost)
    - [**Parameter Tunning**](#parameter-tunning-10)
- [**Voting**](#voting)

#### [Apply Model](#apply-model)



In [105]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, f1_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import plotly.graph_objects as go
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import RFECV
from datetime import date

In [106]:
show_graphs=False

In [107]:
def dataset(x, missing = ""):
    return pd.read_csv('Dataset/' + x + '.csv', sep = ';', low_memory = False, na_values = missing_values).rename(str.strip, axis = 'columns')
    

In [108]:
def abs_min(x):
    return x.abs().min()
abs_min.__name__ = 'abs_min'

def rangev(x):
    return x.max() - x.min()
rangev.__name__ = 'range'


In [109]:
def get_decade(year):
    year_int = int(year)
    string = str(year_int//10) + "0-" + str(year_int//10) + "9"
    return string

In [110]:
def get_month(year):
    string = year[2:4] + "/" + year[0:2]
    return string

In [111]:
def get_balance(year):
    year_int = int(float(year))
    string = "1" + "0"*(len(str(year_int))-1) + "-" + "9" + "9"*(len(str(year_int))-1)
    return string

In [112]:
def calculateAge(birthDate, loanDate):
    loan = list(map(int,loanDate.split("-")))
    birth = list(map(int,birthDate.split("-")))

    loan_date = date(loan[0], loan[1], loan[2])
    

    birth_date = date(birth[0], birth[1], birth[2])
    return (int((loan_date-birth_date).days/365))

Functions for getting number of operations

In [113]:
def count_credit_cash(val):
    return sum(val=="credit in cash")
def count_collect(val):
    return sum(val=="collection from another bank")
def count_with_cash(val):
    return sum(val=="withdrawal in cash")
def count_remi(val):
    return sum(val=="remittance to another bank")
def count_with_card(val):
    return sum(val=="credit card withdrawal")
def count_interest(val):
    return sum(val=="interest credited")

def count_withdrawal(val):
    return sum(val=="withdrawal")
def count_credit(val):
    return sum(val=="credit")

Functions for getting the mean of the operations

In [114]:
def mean_credit_cash(val):
    return np.mean(val=="credit in cash")
def mean_collect(val):
    return np.mean(val=="collection from another bank")
def mean_with_cash(val):
    return np.mean(val=="withdrawal in cash")
def mean_remi(val):
    return np.mean(val=="remittance to another bank")
def mean_with_card(val):
    return np.mean(val=="credit card withdrawal")
def mean_interest(val):
    return np.mean(val=="interest credited")

def mean_withdrawal(val):
    return np.mean(val=="withdrawal")
def mean_credit(val):
    return np.mean(val=="credit")

Functions for getting the standard deviation of the operations

In [115]:
def std_credit_cash(val):
    return np.std(val=="credit in cash")
def std_collect(val):
    return np.std(val=="collection from another bank")
def std_with_cash(val):
    return np.std(val=="withdrawal in cash")
def std_remi(val):
    return np.std(val=="remittance to another bank")
def std_with_card(val):
    return np.std(val=="credit card withdrawal")
def std_interest(val):
    return np.std(val=="interest credited")


def std_withdrawal(val):
    return np.std(val=="withdrawal")
def std_credit(val):
    return np.std(val=="credit")

Functions for getting the covariance of the operations

In [116]:
def cov_credit_cash(val):
    return np.cov(val=="credit in cash")
def cov_collect(val):
    return np.cov(val=="collection from another bank")
def cov_with_cash(val):
    return np.cov(val=="withdrawal in cash")
def cov_remi(val):
    return np.cov(val=="remittance to another bank")
def cov_with_card(val):
    return np.cov(val=="credit card withdrawal")
def cov_interest(val):
    return np.cov(val=="interest credited")

def cov_withdrawal(val):
    return np.cov(val=="withdrawal")
def cov_credit(val):
    return np.cov(val=="credit")

In [117]:
def merge_dataset(loan, district, client, card, trans, account, disp):
    # build train dataset

    df = loan
    df = pd.merge(df, trans, on = 'account_id', suffixes = ('', '_trans'))
    df = pd.merge(df, account, on = 'account_id', suffixes = ('', '_account'))
    #df = df.dropna()
    df = pd.merge(df, district.set_index('code'), left_on = 'district_id', right_index = True, suffixes = ('', '_district'))
    df = pd.merge(df, disp, on = 'account_id', suffixes = ('', '_disp'))
    df = pd.merge(df, card, on = 'disp_id', how = 'outer', suffixes = ('', '_card'))
    df = pd.merge(df, client, on = 'client_id', suffixes = ('', '_client'))
    df = df.drop(['district_id_client'], axis=1)
    df.info()
    return df

In [118]:
def parse_dates(df, date_var, birth):
    loan_dates = [str(int(x))[:2] + "-" + str(int(x))[2:4] + "-" + str(int(x))[4:] for x in df[date_var]]
    return [calculateAge(df[birth][n],loan_dates[n]) for n in range(0,len(df[birth]))]

In [119]:
def parse_date_ranges(df,age):
    age_loan_train = df[age].astype(str)
    return [get_decade(age_loan_train[n]) for n in range(0, len(df))]

In [120]:
def client_dates(client_data):
    # Normalize birthday dates and add a new column (Genre) to distinguish
    birth_dates = client_data['birth_number']
    dates_parsed = []
    genre = []
    for bdate in birth_dates:
        month = int(str(bdate)[2:4])
        if month > 12:
            genre.append(0)
            month = month - 50
            if month < 10:
                month = '0' + str(month)
            else:
                month = str(month)
        else:
            if month < 10:
                month = '0' + str(month)
            else:
                month = str(month)
            genre.append(1)
        dates_parsed.append(str(bdate)[:2] + '-' + month + '-' + str(bdate)[4:])
        
    return dates_parsed, genre

In [121]:
def merge_aggr_dataset(loan, district, client, card, trans, account, disp):
    # build aggregate dataset

    district=district.drop(['name', 'region'], axis=1)

    client["birth_number"], client["genre"]=client_dates(client)

    disp = disp[disp.type.eq('OWNER')]
    disp = disp.drop(['type'], axis=1)

    trans.loc[trans['operation'].isna(), 'operation'] = trans.loc[trans['operation'].isna(), 'k_symbol']
    trans = trans.drop(["k_symbol", "date", "bank", "account"], axis=1)
    trans.loc[trans['type'] == "withdrawal in cash", 'type'] = "withdrawal"
    
    district["unemploymant rate \'95"] = district["unemploymant rate \'95"].fillna(district["unemploymant rate \'95"].median())
    district["no. of commited crimes \'95"] = district["no. of commited crimes \'95"].fillna(district["no. of commited crimes \'95"].mean())

    columns = trans.columns.values.tolist()
    columns.remove("trans_id")
    columns.remove("amount")
    columns.remove("balance")
    columns.remove("operation")
    trans=trans.groupby(columns, as_index=False,group_keys=False).agg({
        "operation": ["count",count_credit_cash, count_collect, count_with_cash, count_remi, count_with_card, count_interest, 
                     mean_credit_cash, mean_collect, mean_with_cash, mean_remi, mean_with_card, mean_interest,
                     std_credit_cash, std_collect, std_with_cash, std_remi, std_with_card, std_interest], 
        "amount": ["mean","min","max","std","last",abs_min,rangev],
        "balance": ["mean","min","max","std","last",abs_min,rangev],
        "type": [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit]
    })
    #cov_withdrawal, cov_credit add these on type
    #cov_credit_cash, cov_collect, cov_with_cash, cov_remi, cov_with_card, cov_interest add these on operation
    trans.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in trans.columns]

    df = loan
    
    df = pd.merge(df, trans, on = 'account_id', suffixes = ('', '_trans'))
    df = pd.merge(df, account, on = 'account_id', suffixes = ('', '_account'))
    df = pd.merge(df, district.set_index('code'), left_on = 'district_id', right_index = True, suffixes = ('', '_district'))
    df = pd.merge(df, disp, on = 'account_id', suffixes = ('', '_disp'))
    df = pd.merge(df, card, on = 'disp_id', how = 'outer', suffixes = ('', '_card'))
    df = pd.merge(df, client, on = 'client_id', suffixes = ('', '_client'))
    df = df.drop(['district_id_client'], axis=1)


    df["age_loan"]=parse_dates(df,"date","birth_number")
    df["age_account"]=parse_dates(df,"date_account","birth_number")
    
    df["age_loan_range"] = parse_date_ranges(df,"age_loan")
    df["age_account_range"] = parse_date_ranges(df,"age_account")
  
    df.drop(['date', 'date_account', 'birth_number'], axis=1, inplace=True)
    df.drop(['card_id', 'type', 'issued'], axis=1, inplace=True)
    
    df.info()
    return df

    
    
    
    
    

In [122]:
missing_values = ['?', 'NA', '']
account_data = dataset('account', missing_values)
client_data = dataset('client', missing_values)
disp_data = dataset('disp', missing_values)
district_data = dataset('district', missing_values)
card_train = dataset('card_train', missing_values)
card_test = dataset('card_test')
loan_train = dataset('loan_train', missing_values)
loan_test = dataset('loan_test')
trans_train = dataset('trans_train', missing_values)
trans_test = dataset('trans_test')

In [123]:
loan_train_aggr, district_data_train_aggr, client_data_train_aggr, card_train_aggr, trans_train_aggr, account_data_train_aggr, disp_data_train_aggr = loan_train.copy(), district_data.copy(), client_data.copy(), card_train.copy(), trans_train.copy(), account_data.copy(), disp_data.copy()

In [124]:
loan_test_aggr, district_data_test_aggr, client_data_test_aggr, card_test_aggr, trans_test_aggr, account_data_test_aggr, disp_data_test_aggr = loan_test.copy(), district_data.copy(), client_data.copy(), card_test.copy(), trans_test.copy(), account_data.copy(), disp_data.copy()

In [125]:
train_data_aggr=merge_aggr_dataset(loan_train_aggr, district_data_train_aggr, client_data_train_aggr, card_train_aggr, trans_train_aggr, account_data_train_aggr, disp_data_train_aggr)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 654 entries, 0 to 653
Data columns (total 67 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   loan_id                                           654 non-null    float64
 1   account_id                                        654 non-null    float64
 2   amount                                            654 non-null    float64
 3   duration                                          654 non-null    float64
 4   payments                                          654 non-null    float64
 5   status                                            654 non-null    float64
 6   operation_count                                   654 non-null    float64
 7   operation_count_credit_cash                       654 non-null    float64
 8   operation_count_collect                           654 non-null    float64
 9   operation_count_with_

In [126]:
test_data_aggr=merge_aggr_dataset(loan_test_aggr, district_data_test_aggr, client_data_test_aggr, card_test_aggr, trans_test_aggr, account_data_test_aggr, disp_data_test_aggr)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707 entries, 0 to 706
Data columns (total 67 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   loan_id                                           707 non-null    int64  
 1   account_id                                        707 non-null    int64  
 2   amount                                            707 non-null    int64  
 3   duration                                          707 non-null    int64  
 4   payments                                          707 non-null    int64  
 5   status                                            0 non-null      float64
 6   operation_count                                   707 non-null    int64  
 7   operation_count_credit_cash                       707 non-null    int64  
 8   operation_count_collect                           707 non-null    int64  
 9   operation_count_with_

In [127]:
train_data_aggr

,loan_id,account_id,amount,duration,payments,status,operation_count,operation_count_credit_cash,operation_count_collect,operation_count_with_cash,operation_count_remi,operation_count_with_card,operation_count_interest,operation_mean_credit_cash,operation_mean_collect,operation_mean_with_cash,operation_mean_remi,operation_mean_with_card,operation_mean_interest,operation_std_credit_cash,operation_std_collect,operation_std_with_cash,operation_std_remi,operation_std_with_card,operation_std_interest,amount_mean,amount_min,amount_max,amount_std,amount_last,amount_abs_min,amount_range,balance_mean,balance_min,balance_max,balance_std,balance_last,balance_abs_min,balance_range,type_count_withdrawal,type_count_credit,type_mean_withdrawal,type_mean_credit,type_std_withdrawal,type_std_credit,district_id,frequency,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,disp_id,client_id,genre,age_loan,age_account,age_loan_range,age_account_range
0,5314.0,1787.0,96396.0,12.0,8033.0,-1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,5025.000000,1100.0,9900.0,3774.806838,3300.0,1100.0,8800.0,12250.000000,1100.0,20100.0,8330.866301,20100.0,1100.0,19000.0,0.0,4.0,0.0,1.0,0.0,0.0,30.0,weekly issuance,94812.0,15.0,13.0,8.0,2.0,10.0,81.8,9650.0,3.38,3.67,100.0,2985.0,2804.0,2166,2166.0,0,45,45,40-49,40-49
1,6903.0,9337.0,52128.0,24.0,2172.0,1.0,39.0,13.0,0.0,0.0,0.0,0.0,26.0,0.333333,0.0,0.000000,0.000000,0.0,0.666667,0.471405,0.0,0.000000,0.000000,0.0,0.471405,6244.838462,38.7,44473.0,12442.414306,84.4,38.7,44434.3,38757.382051,247.8,59944.2,20171.556113,58744.2,247.8,59696.4,0.0,39.0,0.0,1.0,0.0,0.0,30.0,monthly issuance,94812.0,15.0,13.0,8.0,2.0,10.0,81.8,9650.0,3.38,3.67,100.0,2985.0,2804.0,11189,11497.0,0,34,34,30-39,30-39
2,6903.0,9337.0,52128.0,24.0,2172.0,1.0,32.0,0.0,0.0,32.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,5771.993750,0.1,27300.0,6910.801439,14.6,0.1,27299.9,27003.106250,144.2,58844.8,18276.300023,58844.8,144.2,58700.6,32.0,0.0,1.0,0.0,0.0,0.0,30.0,monthly issuance,94812.0,15.0,13.0,8.0,2.0,10.0,81.8,9650.0,3.38,3.67,100.0,2985.0,2804.0,11189,11497.0,0,34,34,30-39,30-39
3,5316.0,1801.0,165960.0,36.0,4610.0,1.0,17.0,12.0,0.0,0.0,0.0,0.0,5.0,0.705882,0.0,0.000000,0.000000,0.0,0.294118,0.455645,0.0,0.000000,0.000000,0.0,0.455645,13523.158824,2.9,36574.0,13998.092225,29171.0,2.9,36571.1,53287.205882,700.0,120512.8,34755.252013,55627.9,700.0,119812.8,0.0,17.0,0.0,1.0,0.0,0.0,46.0,monthly issuance,112709.0,48.0,20.0,7.0,3.0,10.0,73.5,8369.0,1.79,2.31,117.0,2854.0,2618.0,2181,2181.0,1,24,24,20-29,20-29
4,5316.0,1801.0,165960.0,36.0,4610.0,1.0,20.0,0.0,0.0,12.0,8.0,0.0,0.0,0.000000,0.0,0.600000,0.400000,0.0,0.000000,0.000000,0.0,0.489898,0.489898,0.0,0.000000,8884.240000,14.6,54300.0,14074.800608,3419.0,14.6,54285.4,51061.015000,23795.0,103028.2,24234.615159,52208.9,23795.0,79233.2,20.0,0.0,1.0,0.0,0.0,0.0,46.0,monthly issuance,112709.0,48.0,20.0,7.0,3.0,10.0,73.5,8369.0,1.79,2.31,117.0,2854.0,2618.0,2181,2181.0,1,24,24,20-29,20-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
649,5606.0,3084.0,253512.0,36.0,7042.0,-1.0,15.0,0.0,0.0,15.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,10556.000000,30.0,30200.0,9618.492524,8100.0,30.0,30170.0,32607.700000,4418.4,80844.2,19782.583028,15218.4,4418.4,

In [128]:
test_data_aggr

,loan_id,account_id,amount,duration,payments,status,operation_count,operation_count_credit_cash,operation_count_collect,operation_count_with_cash,operation_count_remi,operation_count_with_card,operation_count_interest,operation_mean_credit_cash,operation_mean_collect,operation_mean_with_cash,operation_mean_remi,operation_mean_with_card,operation_mean_interest,operation_std_credit_cash,operation_std_collect,operation_std_with_cash,operation_std_remi,operation_std_with_card,operation_std_interest,amount_mean,amount_min,amount_max,amount_std,amount_last,amount_abs_min,amount_range,balance_mean,balance_min,balance_max,balance_std,balance_last,balance_abs_min,balance_range,type_count_withdrawal,type_count_credit,type_mean_withdrawal,type_mean_credit,type_std_withdrawal,type_std_credit,district_id,frequency,no. of inhabitants,no. of municipalities with inhabitants < 499,no. of municipalities with inhabitants 500-1999,no. of municipalities with inhabitants 2000-9999,no. of municipalities with inhabitants >10000,no. of cities,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,disp_id,client_id,genre,age_loan,age_account,age_loan_range,age_account_range
0,5895,4473,93960,60,1566,NaN,30,16,0,0,0,0,14,0.533333,0.000000,0.000000,0.000000,0.0,0.466667,0.498888,0.000000,0.000000,0.000000,0.0,0.498888,12769.220000,82.2,36637.5,14018.915135,273.0,82.2,36555.3,56845.746667,800.0,88246.7,17910.038799,69263.1,800.0,87446.7,0,30,0.0,1.0,0.0,0.0,45,monthly issuance,77917,85,19,6,1,5,53.5,8390,2.28,2.89,132,2080.0,2122,5395,5395,1,34,33,30-39,30-39
1,5895,4473,93960,60,1566,NaN,59,0,0,40,19,0,0,0.000000,0.000000,0.677966,0.322034,0.0,0.000000,0.000000,0.000000,0.467256,0.467256,0.0,0.000000,5653.016949,14.6,30000.0,6378.603783,19700.0,14.6,29985.4,53337.722034,35018.4,76163.2,11091.606565,49548.5,35018.4,41144.8,59,0,1.0,0.0,0.0,0.0,45,monthly issuance,77917,85,19,6,1,5,53.5,8390,2.28,2.89,132,2080.0,2122,5395,5395,1,34,33,30-39,30-39
2,5172,1083,50976,36,1416,NaN,38,22,0,0,0,0,16,0.578947,0.000000,0.000000,0.000000,0.0,0.421053,0.493728,0.000000,0.000000,0.000000,0.0,0.493728,6173.323684,50.2,17388.0,6301.712590,11592.0,50.2,17337.8,30416.542105,400.0,48735.9,10545.032128,26499.1,400.0,48335.9,0,38,0.0,1.0,0.0,0.0,45,monthly issuance,77917,85,19,6,1,5,53.5,8390,2.28,2.89,132,2080.0,2122,1300,1300,1,18,17,10-19,10-19
3,5172,1083,50976,36,1416,NaN,85,0,0,47,38,0,0,0.000000,0.000000,0.552941,0.447059,0.0,0.000000,0.000000,0.000000,0.497189,0.497189,0.0,0.000000,2509.096471,14.6,19200.0,2789.817929,4351.0,14.6,19185.4,26992.449412,12229.1,47900.9,8300.079414,21313.1,12229.1,35671.8,85,0,1.0,0.0,0.0,0.0,45,monthly issuance,77917,85,19,6,1,5,53.5,8390,2.28,2.89,132,2080.0,2122,1300,1300,1,18,17,10-19,10-19
4,6207,5912,184620,60,3077,NaN,43,4,20,0,0,0,19,0.093023,0.465116,0.000000,0.000000,0.0,0.441860,0.290465,0.498782,0.000000,0.000000,0.0,0.496608,9867.893023,21.5,27692.0,10279.615313,27692.0,21.5,27670.5,38328.760465,1100.0,65517.7,11715.137668,52423.4,1100.0,64417.7,0,43,0.0,1.0,0.0,0.0,45,monthly issuance,77917,85,19,6,1,5,53.5,8390,2.28,2.89,132,2080.0,2122,7150,7150,0,34,33,30-39,30-39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,6469,7180,99744,24,4156,NaN,60,0,0,47,13,0,0,0.000000,0.000000,0.783333,0.216667,0.0,0.000000,0.000000,0.000000,0.411974,0.411974,0.0,0.000000,6774.050000,30.0,26500.0,6058.416576,8089.0,30.0,26470.0,38044.693333,15075.3,71878.9,14477.953958,71878.9,15075.3,56803.6,60,0,1.0,0.0,0.0,0.0,7,weekly issuance,94725,38,28,1,3,6,63.4,9920,2.26,2.87,130,4289.0,4846,8666,8666,0,27,25,20-29,20-29
703,5614,3137,253560,60,4226,NaN,43,2,21,0,0,0,20,0.046512,0.488372,0.000000,0.00

We can see the distribution of the district regions for each one of the different regions.

In [ ]:

if show_graphs:
    plt.figure(figsize=(15,8))
    sb.countplot(x='region', data=district_data)
    plt.title("Number of districts per region", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.xlabel('Region');
    plt.ylabel('Number of districts from region');
    plt.show()

To study if the region might have an impact on the acceptance of the loan we will create a graph comparing the percentages of successful loans per region.

In [ ]:
if show_graphs:
    district_data_region = district_data.copy()
    region_data = loan_train
    region_data = pd.merge(region_data, trans_train, on = 'account_id', suffixes = ('', '_trans'))
    region_data = pd.merge(region_data, account_data, on = 'account_id', suffixes = ('', '_account'))
    #train_data = train_data.dropna()
    region_data = pd.merge(region_data, district_data_region.set_index('code'), left_on = 'district_id', right_index = True, suffixes = ('', '_district'))

    region_total = region_data["region"].value_counts()
    tuples_total = [tuple((x, y)) for x, y in region_total.items()]

    regions_status_1 = region_data.loc[region_data['status'] == 1]
    region_total_1 = regions_status_1["region"].value_counts()
    tuples_total_1 = [tuple((x, y)) for x, y in region_total_1.items()]

    lista=[]
    for x in tuples_total:
        for y in tuples_total_1:
            if x[0]==y[0]:
                lista.append((x[0],x[1],y[1]))

    percentages = [(i[0],i[2] / i[1]) for i in lista]
    percentages.sort(key = lambda x: -x[1])

    x = [i[0] for i in percentages]
    y = [i[1] for i in percentages]

    plt.figure(figsize=(15,8))
    sb.barplot(x,y)
    plt.ylim(0.7, 1)
    plt.title("Percentage of successful loans per region", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.xlabel('Region');
    plt.ylabel('Percentage of successful loans');
    plt_show()


Now let's see the different values the parameter type can have.

In [ ]:
if show_graphs:
    disp_data_pie = disp_data.copy()

    ser = disp_data_pie.groupby('type')['type'].count()
    ser = ser.sort_values(ascending=False)
    ser = ser.iloc[[0,1]]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.pie(ser.values, labels=ser.index, startangle=90, autopct=lambda x:int(x/100.*ser.sum()+0.1), pctdistance=0.8, counterclock=False)
    ax.legend()
    plt.axis('equal')
    plt.title('Type of disposition', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    fig.set_size_inches(15, 5)

Since only owner can issue permanent orders and ask for a loan, there is no interess in keeping this parameter in our dataset.

In [ ]:
# Removing name and region from district
district_data.drop(['name', 'region'], axis=1, inplace=True)

# only owner can issue permanent orders and ask for a loan
disp_owners = disp_data[disp_data.type.eq('OWNER')]
disp_owners.drop(['type'], axis=1, inplace=True)

As *birth_number* column doesn't add any value to our model as it is in the original data (it's represented as an int in format YYMMDD for men and YYMM+50DD for women), let's put it in format YY-MM-DD and create a new column called *genre*.

In [ ]:
# Normalize birthday dates and add a new column (Genre) to distinguish

birth_dates = client_data['birth_number']
dates_parsed = []
genre = []
for bdate in birth_dates:
    month = int(str(bdate)[2:4])
    if month > 12:
        genre.append(0)
        month = month - 50
        if month < 10:
            month = '0' + str(month)
        else:
            month = str(month)
    else:
        #print('AAAA: ' + str(month))
        if month < 10:
            month = '0' + str(month)
            #print('BBBB: ' + str(month))
        else:
            month = str(month)
        genre.append(1)
    dates_parsed.append(str(bdate)[:2] + '-' + month + '-' + str(bdate)[4:])
    

# client_data = client_data.drop(['birth_number'], axis = 1)
client_data['birth_number'] = dates_parsed
client_data['genre'] = genre
client_data.head()

We can also see the *genre* distribution, where 1 means male and 0 female.

In [ ]:
if show_graphs:
    client_data_genre_copy = client_data.copy()

    ser = client_data_genre_copy.groupby('genre')['genre'].count()
    ser = ser.sort_values(ascending=False)
    ser = ser.iloc[[0,1]]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.pie(ser.values, labels=["Male","Female"], startangle=90, autopct=lambda x:int(x/100.*ser.sum()+0.1), pctdistance=0.8, counterclock=False)
    ax.legend()
    plt.axis('equal')
    plt.title('Genre distribution', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    fig.set_size_inches(15, 5)

In [ ]:
train_data=merge_dataset(loan_train, district_data, client_data, card_train, trans_train, account_data, disp_owners)

In [ ]:
train_data_aggr=merge_aggr_dataset(loan_train, district_data, client_data, card_train, trans_train, account_data, disp_owners)

In [ ]:
train_data_aggr

In [ ]:
test_data=merge_dataset(loan_test, district_data, client_data, card_test, trans_test, account_data, disp_owners)

In [ ]:
test_data.head()

Now we can create a column *age_loan* where we will keep the age of the client at the time of the requested loan.

In [ ]:
loan_dates = train_data['date']

loan_dates = [str(int(x))[:2] + "-" + str(int(x))[2:4] + "-" + str(int(x))[4:] for x in loan_dates]

birth_dates = train_data['birth_number']

#print(sorted(birth_dates))

age_list = [calculateAge(birth_dates[n],loan_dates[n]) for n in range(0,len(birth_dates))]

train_data["age_loan"] = age_list

# print(train_data["age_loan"])


Now let's do the same for the test data.

In [ ]:
loan_dates = test_data['date']

loan_dates = [str(int(x))[:2] + "-" + str(int(x))[2:4] + "-" + str(int(x))[4:] for x in loan_dates]

birth_dates = test_data['birth_number']

age_list = [calculateAge(birth_dates[n],loan_dates[n]) for n in range(0,len(test_data))]

test_data["age_loan"] = age_list

# print(test_data["age_loan"])

Also, let's create a new column that calculates the age of each client when he creates the account (*age_account*).

In [ ]:
account_dates_train = train_data['date_account']
account_dates_test = test_data['date_account']

account_dates_train = [str(int(x))[:2] + "-" + str(int(x))[2:4] + "-" + str(int(x))[4:] for x in account_dates_train]
account_dates_test = [str(int(x))[:2] + "-" + str(int(x))[2:4] + "-" + str(int(x))[4:] for x in account_dates_test]

age_list_train = [calculateAge(birth_dates[n],account_dates_train[n]) for n in range(0,len(train_data))]
age_list_test = [calculateAge(birth_dates[n],account_dates_test[n]) for n in range(0,len(test_data))]

train_data['age_account'] = age_list_train
test_data['age_account'] = age_list_test

# print(train_data["age_account"])

Now, let's simplify *age_loan* and *age_account* values, converting them into age ranges divided by decades.

In [ ]:
age_loan_train = train_data['age_loan'].astype(str)
age_loan_test = test_data['age_loan'].astype(str)

age_loan_train = [get_decade(age_loan_train[n]) for n in range(0, len(train_data))]
age_loan_test = [get_decade(age_loan_test[n]) for n in range(0, len(test_data))]

train_data['age_loan_range'] = age_loan_train
test_data['age_loan_range'] = age_loan_test

train_data.head(10)

In [ ]:
age_loan_train = train_data['age_account'].astype(str)
age_loan_test = test_data['age_account'].astype(str)

age_loan_train = [get_decade(age_loan_train[n]) for n in range(0, len(train_data))]
age_loan_test = [get_decade(age_loan_test[n]) for n in range(0, len(test_data))]

train_data['age_account_range'] = age_loan_train
test_data['age_account_range'] = age_loan_test

train_data.head(10)

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    train_data_copy = train_data_copy.astype({'age_loan': str})

    train_data_copy['age_loan'] = train_data_copy['age_loan'].apply(lambda x: get_decade(x))

    train_data_copy = train_data_copy.sort_values('age_loan')

    fig = plt.figure()
    plt.figure(figsize=(15,8))
    plt.title("Age by decade at the date of the loan", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    ax = sb.countplot(x="age_loan", data=train_data_copy)

    plt.show()

Now we can see the months when the accounts were created.

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy = train_data_copy.sort_values('date_account')
    train_data_copy = train_data_copy.astype({'date_account': str})

    train_data_copy['date_account'] = train_data_copy['date_account'].apply(lambda x: get_month(x[0:4]))

    fig = plt.figure()
    plt.figure(figsize=(25,20))
    plt.title("Date of creation of the account per month")
    ax = sb.countplot(x="date_account", data=train_data_copy)

    plt.show()

We can also see the month and year of when the loans were created and their relation to the number of successful loans.

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy = train_data_copy.sort_values('date')
    train_data_copy = train_data_copy.astype({'date': str})

    train_data_copy['date'] = train_data_copy['date'].apply(lambda x: get_month(x[0:4]))

    train_data_copy = train_data_copy.astype({'status': str})
    train_data_copy['status'] = train_data_copy['status'].replace(['1.0','-1.0'],['Successful Loan','Unsuccessful Loan'])

    fig = plt.figure()
    plt.figure(figsize=(25,20))
    plt.title("Number of successful loans per month", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    ax = sb.countplot(x ='date', hue = "status", data = train_data_copy)

    plt.show()

As follows we can see the months when the most transfers were taken.

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy = train_data_copy.sort_values('date_trans')
    train_data_copy = train_data_copy.astype({'date_trans': str})

    train_data_copy['date_trans'] = train_data_copy['date_trans'].apply(lambda x: get_month(x[0:4]))

    fig = plt.figure()
    plt.figure(figsize=(25,20))
    plt.title("Months with the most tranfers", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    ax = sb.countplot(x="date_trans", data=train_data_copy)
    plt.show()

Now that we have taken all informations that we need from all dates, we can remove these columns from our dataset.

In [ ]:
train_data.drop(['date', 'date_trans', 'date_account', 'birth_number'], axis=1, inplace=True)
test_data.drop(['date', 'date_trans', 'date_account', 'birth_number'], axis=1, inplace=True)

In [ ]:
if show_graphs:
    # Null values for each attribute
    train_data.isnull().sum().plot(kind='bar', figsize=(18,8), fontsize=14,);
    plt.title("Number of null variables per collumn", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.ylabel('Null values');

We can remove *bank* and *account* columns, as these columns only record the destination account.

In [ ]:
train_data.drop(['bank', 'account'], axis=1, inplace=True)
test_data.drop(['bank', 'account'], axis=1, inplace=True)

Let's look at the possible values and the respective frequency for some columns.

In [ ]:
print(train_data.operation.value_counts())
print('Null values: ' + str(train_data.operation.isnull().sum()))
print()
print(train_data.k_symbol.value_counts())
print('Null values: ' + str(train_data.k_symbol.isnull().sum()))
print()
print(train_data.type_card.value_counts())
print('Null values: ' + str(train_data.type_card.isnull().sum()))
print()
print(train_data.type.value_counts())
print('Null values: ' + str(train_data.type.isnull().sum()))
print()

In [ ]:
train_data_copy = train_data.copy()

train_data_copy = train_data_copy.loc[:, train_data_copy.columns.intersection(['type','operation'])]
train_data_copy["operation"] = train_data_copy["operation"].fillna('NaN')

train_data_copy.sort_values("operation")

print(train_data_copy["type"].unique())
print(train_data_copy["operation"].unique())

In [ ]:
if show_graphs:
    sb.displot(train_data_copy,x='operation', y='type', aspect=3)
    plt.xlabel('Operation');
    plt.ylabel('Type');
    plt.title("Comparition between Operation and Type", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

As we can see all the rows with operation *credit in cash* and *collection from another bank* are of type credit. All the *withdrawal in cash*, *remittance to another bank* and *credit cash withdrawal* operations are of the type *withdraw* or *withdrawal with cash* (in the case of the homonymous operation)

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    print(train_data_copy.operation.value_counts())
    train_data_copy = train_data_copy.groupby(['k_symbol', 'operation'])['operation'].count()
    print(train_data_copy)

    plt.rcParams["figure.figsize"] = (15,10)
    train_data_copy.plot(kind='bar', stacked=True)
    plt.xlabel('K_symbol');
    plt.ylabel('Quantity');
    plt.title("Number of operation types per k_symbol", fontdict={'fontsize': 14, 'fontweight': 'bold'})

    plt.show()

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy["operation"]=train_data_copy["operation"].fillna("Not Available")
    train_data_copy["k_symbol"]=train_data_copy["k_symbol"].fillna("Not Available")
    train_data_copy['k_symbol'] = train_data_copy['k_symbol'].replace([' '],['Not Available'])
    train_data_copy['operation'] = train_data_copy['operation'].replace([''],['Not Available'])

    fig = plt.figure()

    sb.catplot(x="operation", hue="k_symbol", kind="count",
                palette="pastel", edgecolor=".6",
                data=train_data_copy, height=10, aspect=1.5)
    plt.title('Relation between k_symbols and operation', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.ylim(0)
    plt.rcParams['figure.figsize']=(20,10)

    plt.show()


As we can see the great majority of the k_symbol types are directly related to a certain operation type

Looking for **type** column, let's convert all *withdrawal in cash* occurrences into *withdrawal*, as it is basically the same. 

In [ ]:
train_data.loc[train_data['type'] == "withdrawal in cash", 'type'] = "withdrawal"
test_data.loc[test_data['type'] == "withdrawal in cash", 'type'] = "withdrawal"

Now, looking at *operation* and *k_symbol* columns, let's select all rows that have null value in *operation* and look at the respective value in *k_symbol* value.

In [ ]:
operations_null = train_data[train_data['operation'].isnull()]
operations_null.k_symbol.value_counts()

All rows that have **operation** column with null value have *interested credited* in *k_symbol* column. We can try to replace the null values of *operation* column with the value that is in *k_symbol* column and then delete *k_symbol* parameter.

In [ ]:
train_data.loc[train_data['operation'].isna(), 'operation'] = train_data.loc[train_data['operation'].isna(), 'k_symbol']
test_data.loc[test_data['operation'].isna(), 'operation'] = test_data.loc[test_data['operation'].isna(), 'k_symbol']

train_data.drop(['k_symbol'], axis=1, inplace=True)
test_data.drop(['k_symbol'], axis=1, inplace=True)

Representation of the Card Type, with NaN values

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    train_data_copy['type_card'] = train_data_copy['type_card'].fillna("NaN")

    ser = train_data_copy.groupby('type_card')['type_card'].count()
    ser = ser.sort_values(ascending=False)
    ser = ser.iloc[[0,1,2,3]]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.pie(ser.values, labels=['','','',''], startangle=90, autopct=lambda x:int(x/100.*ser.sum()+0.1), pctdistance=0.8, counterclock=False)
    ax.legend(loc=3, labels=ser.index)
    plt.axis('equal')
    plt.title('Card Type distribution', y=1.05, fontsize=20)
    fig.set_size_inches(15, 5)

Representation of the Card Type, without NaN values

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    ser = train_data_copy.groupby('type_card')['type_card'].count()
    ser = ser.sort_values(ascending=False)
    ser = ser.iloc[[0,1,2]]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.pie(ser.values, labels=ser.index, startangle=90, autopct=lambda x:int(x/100.*ser.sum()+0.1), pctdistance=0.8, counterclock=False)
    ax.legend()
    plt.axis('equal')
    plt.title('Card Type distribution', y=1.05, fontsize=20)
    fig.set_size_inches(15, 5)

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    train_data_copy['type_card'] = train_data_copy['type_card'].fillna("NaN")

    ser = train_data_copy.groupby('type_card')['type_card'].count()
    ser = ser.sort_values(ascending=False)
    ser = ser.iloc[[0,1,2,3]]

    fig = plt.figure()
    ax = fig.add_subplot(111)

    ax.pie(ser.values, labels=['','','',''], startangle=90, autopct=lambda x:int(x/100.*ser.sum()+0.1), pctdistance=0.8, counterclock=False)
    ax.legend(loc=3, labels=ser.index)
    plt.axis('equal')
    plt.title('Card Type distribution', y=1.05, fontsize=20)
    fig.set_size_inches(15, 5)

[back](#index)
#### District Data <a id="district-data"></a>

In [ ]:
if show_graphs:
    district_scatter_plot = sb.PairGrid(district_data)
    district_scatter_plot.map(plt.scatter)

In [ ]:
print('Null values unemploymant rate in \'95:' + str(district_data['unemploymant rate \'95'].isnull().sum()))
print()
print('Null values no. of commited crimes \'95 :' + str(district_data['no. of commited crimes \'95'].isnull().sum()))
print()

In [ ]:
if show_graphs:
    plt.figure(figsize=(20,10))
    plt.title('Distribution of district\'s unemploymant rate in \'95', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.hist(district_data['unemploymant rate \'95'])

Representation of the Card Type, without NaN values

In [ ]:
if show_graphs:
    plt.figure(figsize=(20,10))
    plt.title('Distribution of district\'s no. of commited crimes \'95', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.hist(district_data['no. of commited crimes \'95'])

In [ ]:
test_data.info()

In [ ]:
# Fill null values in district's unemploymant rate in '95 and district's no. of commited crimes '95
# See if it is better to use median or mean 

train_data['unemploymant rate \'95'].fillna(train_data['unemploymant rate \'95'].median(), inplace=True)

train_data['no. of commited crimes \'95'].fillna(train_data['no. of commited crimes \'95'].mean(), inplace=True)

test_data['unemploymant rate \'95'].fillna(test_data['unemploymant rate \'95'].median(), inplace=True)

test_data['no. of commited crimes \'95'].fillna(test_data['no. of commited crimes \'95'].mean(), inplace=True)

train_data_no_dummies = train_data.copy()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

[back](#index)
#### Dataset Managing <a id="dataset"></a>

In [ ]:
# needs to be checked
option = 0
if option == 0:
    train_data.drop(['issued'], axis=1, inplace=True)
    test_data.drop(['issued'], axis=1, inplace=True)
if option == 1:
    train_data.drop(['operation', 'issued'], axis=1, inplace=True)
    test_data.drop(['operation', 'issued'], axis=1, inplace=True)
# dropped type and issued
elif option == 2:
    train_data.drop(['issued', 'type'], axis=1, inplace=True)
    test_data.drop(['issued', 'type'], axis=1, inplace=True)
#doesn't work
elif option == 4:
    train_data.drop(["card_id","type_card", "issued","client_id", "disp_id"],axis=1,inplace=True)
    test_data.drop(["card_id","type_card", "issued","client_id", "disp_id"],axis=1,inplace=True)
    columns = train_data.columns.values.tolist()
    columns.remove("trans_id")
    columns.remove("amount_trans")
    columns.remove("balance")
    columns.remove("operation")
    columns.remove("type")   
    columns.remove("age_account")  
    columns.remove("age_account_range")  
    
    train_data=train_data.groupby(columns, as_index=False,group_keys=False).agg({
        "operation": ["count",count_credit_cash, count_collect, count_with_cash, count_remi, count_with_card, count_interest, 
                     mean_credit_cash, mean_collect, mean_with_cash, mean_remi, mean_with_card, mean_interest,
                     std_credit_cash, std_collect, std_with_cash, std_remi, std_with_card, std_interest,
                     cov_credit_cash, cov_collect, cov_with_cash, cov_remi, cov_with_card, cov_interest],
        "amount_trans": ["mean","min","max","std","last",abs_min,rangev],
        "balance": ["mean","min","max","std","last",abs_min,rangev],
        "type": [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit, cov_withdrawal, cov_credit]
        
    })
    
    test_data=test_data.groupby(columns, as_index=False,group_keys=False).agg({
        "operation": ["count",count_credit_cash, count_collect, count_with_cash, count_remi, count_with_card, count_interest, 
                     mean_credit_cash, mean_collect, mean_with_cash, mean_remi, mean_with_card, mean_interest,
                     std_credit_cash, std_collect, std_with_cash, std_remi, std_with_card, std_interest,
                     cov_credit_cash, cov_collect, cov_with_cash, cov_remi, cov_with_card, cov_interest],
        "amount_trans": ["mean","min","max","std","last",abs_min,rangev],
        "balance": ["mean","min","max","std","last",abs_min,rangev],
        "type": [count_withdrawal, count_credit, mean_withdrawal, mean_credit, std_withdrawal, std_credit, cov_withdrawal, cov_credit]
        
    })
    train_data.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in train_data.columns]
    test_data.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in test_data.columns]
    
    


In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
train_data = pd.get_dummies(train_data, columns=['frequency'], dtype=bool)
test_data = pd.get_dummies(test_data, columns=['frequency'], dtype=bool)

if option != 4:
    train_data = pd.get_dummies(train_data, columns=['type_card'], dtype = bool)
    test_data = pd.get_dummies(test_data, columns=['type_card'], dtype = bool)
    
train_data = pd.get_dummies(train_data, columns=['age_loan_range'], dtype = bool)
test_data = pd.get_dummies(test_data, columns=['age_loan_range'], dtype = bool)

train_data = pd.get_dummies(train_data, columns=['age_account_range'], dtype = bool)
test_data = pd.get_dummies(test_data, columns=['age_account_range'], dtype = bool)

if (option != 2 and option != 4):
    train_data = pd.get_dummies(train_data, columns=['type'], dtype = bool)
    test_data = pd.get_dummies(test_data, columns=['type'], dtype = bool)

if( option != 1 and option != 4) :
    train_data = pd.get_dummies(train_data, columns=['operation'], dtype = bool)
    test_data = pd.get_dummies(test_data, columns=['operation'], dtype = bool)

# test_data = test_data.drop_duplicates(subset=['loan_id'], keep='first')

train_data.info()

In [ ]:
competition_inputs = test_data.drop(columns=["loan_id", "status"])
# test_data = test_data.drop(columns=["status"])
all_ids_comp = test_data['loan_id'].values

#### [back](#index)
#### Data Exploration <a class="anchor" id="data-exploration"></a>

After the previous changes we end up with the following columns to study

In [ ]:
test_data.info()

In [ ]:
if show_graphs:
    plt.figure(figsize=(15,10))
    ax = sb.barplot(train_data["duration"], train_data["amount"])
    plt.xlabel('Duration');
    plt.ylabel('Amount');
    plt.title('Loan duration compared the amount', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

We can clearly see that with bigger loan amounts the time taken to repay them is exponentially higher 

In [ ]:
if show_graphs:
    plt.rcParams["figure.figsize"] = (15,10)
    ax = plt.plot(train_data["amount"], train_data["payments"], linestyle='none', marker='o', alpha=0.3)
    plt.title('Number of payments per different amount', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.xlabel('Amount');
    plt.ylabel('Payments');
    plt.show()

As we can check by the values in the graph, it seems that the number of payments is defined by the bank since the results are very linear and follow a line according to the amount loaned

In [ ]:
if show_graphs:
    plt.figure(figsize=(20,10))
    #ax = sb.barplot(train_data["amount_trans"], train_data["balance"])
    plt.plot(train_data["balance_mean"],train_data["amount_trans_mean"], 'o', color='green');
    plt.title('Distribution of tranfer amount by balance', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.ylabel('Transfer Amount');
    plt.xlabel('Balance');

    plt.show()

As we can see as the balance increases the transfer amount also increase and is rarely higher than the balance. It can also be seen that a straight continous line exists with gradient 1. This means the amount of some transfers are actually correspondent, or close, to the total amount in the account.

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy = train_data_copy.astype({'status': str})
    train_data_copy['status'] = train_data_copy['status'].replace(['1.0','-1.0'],['Successful Loan','Unsuccessful Loan'])

    train_data_copy = train_data_copy.groupby(['no. of cities', 'status'])['status'].count().unstack().fillna(0)

    plt.rcParams["figure.figsize"] = (15,10)
    train_data_copy.plot(kind='bar', stacked=True)
    plt.title("Number of cities per district and loan success rate", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    train_data_copy = train_data_copy.astype({'status': str})
    train_data_copy['status'] = train_data_copy['status'].replace(['1.0','-1.0'],['Successful Loan','Unsuccessful Loan'])

    fig = plt.figure()
    sb.violinplot(y=train_data_copy["no. of inhabitants"], x=train_data_copy["status"])
    plt.title('Influence of number of inhabitants in loan success', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.ylim(0)
    fig.set_size_inches(15, 15)
    plt.show()

In [ ]:
#print(sorted(train_data["no. of inhabitants"]))

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    train_data_copy = train_data.copy()
    train_data_copy = train_data_copy.astype({'age_loan': str})

    train_data_copy['age_loan'] = train_data_copy['age_loan'].apply(lambda x: get_decade(x))

    train_data_copy = train_data_copy.astype({'status': str})
    train_data_copy['status'] = train_data_copy['status'].replace(['1.0','-1.0'],['Successful Loan','Unsuccessful Loan'])

    train_data_copy = train_data_copy.groupby(['age_loan', 'status'])['status'].count().unstack().fillna(0)

    plt.rcParams["figure.figsize"] = (15,10)
    train_data_copy.plot(kind='bar', stacked=True)
    plt.title("Loan success rate per decade of age", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()

    train_data_copy = train_data_copy.astype({'status': str})
    train_data_copy['status'] = train_data_copy['status'].replace(['1.0','-1.0'],['Successful Loan','Unsuccessful Loan'])

    train_data_copy = train_data_copy.groupby(['genre', 'status'])['status'].count().unstack().fillna(0)

    plt.rcParams["figure.figsize"] = (15,10)
    train_data_copy.plot(kind='bar', stacked=True)
    plt.title("Loan success rate per decade of genre", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    plt.figure(figsize=(15,10))

    train_data_names = train_data.copy()
    train_data_names["genre"] = train_data_names["genre"].astype(str)
    train_data_names["genre"].replace({"1": "Male" , "0": "Female"}, inplace=True)

    sb.scatterplot(train_data_names["amount"],train_data_names["age_loan"],train_data_names["genre"], alpha=0.5, sizes=(10, 1000), hue="time")
    plt.xlabel("Amount")
    plt.ylabel("Age")
    plt.title("Amount by age and genre", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    train_data_names = train_data.copy()

    train_data_names = train_data_names.astype({'age_loan': str})

    train_data_names['age_loan'] = train_data_names['age_loan'].apply(lambda x: get_decade(x))

    train_data_names = train_data_names.sort_values('age_loan')

    plt.figure(figsize=(15,10))

    sb.boxplot(x=train_data_names["age_loan"], y=train_data_names["amount"])
    plt.title("Loan amount by age", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    train_data_ages = train_data_no_dummies.copy()

    train_data_ages = train_data_ages.astype({'age_loan': str})

    train_data_ages['age_loan'] = train_data_ages['age_loan'].apply(lambda x: get_decade(x))

    train_data_ages = train_data_ages.astype({'age_account': str})

    train_data_ages['age_account'] = train_data_ages['age_account'].apply(lambda x: get_decade(x))

    train_data_names = train_data_ages.sort_values('age_loan')
    sb.barplot(x='age_account', y='amount', hue='age_loan', data=train_data_ages, saturation=0.8)
    plt.title("Date of creation of account compared to amount and age when the loan was made", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    train_data_ages = train_data_no_dummies.copy()

    train_data_ages = train_data_ages.astype({'age_loan': str})

    train_data_ages['age_loan'] = train_data_ages['age_loan'].apply(lambda x: get_decade(x))

    train_data_balance = train_data_ages.astype({'balance': str})

    train_data_balance['balance'] = train_data_balance['balance'].apply(lambda x: get_balance(x))

    train_data_balance = train_data_balance.sort_values('balance')
    sb.boxplot(x='balance', y='amount', hue='age_loan', data=train_data_balance, saturation=0.8)
    plt.title("Balance and amount of loan by age", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

In [ ]:
if show_graphs:
    plt.figure(figsize=(15,10))

    train_data_names = train_data.copy()
    train_data_names["genre"] = train_data_names["genre"].astype(str)
    train_data_names["genre"].replace({"1": "Male" , "0": "Female"}, inplace=True)

    sb.scatterplot(train_data_names["average salary"],train_data_names["age_loan"],train_data_names["district_id"], alpha=0.5, sizes=(10, 1000), hue="time")
    plt.xlabel("Average Salary")
    plt.ylabel("Age")
    plt.title("Average Salary by age and district", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    plt.show()

As follows we can see the average salary by age

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy = train_data_copy.astype({'age_loan': str})

    train_data_copy['age_loan'] = train_data_copy['age_loan'].apply(lambda x: get_decade(x))

    train_data_copy = train_data_copy.sort_values('age_loan')
    train_data_copy["average salary"] = train_data_copy["average salary"].astype(float)

    fig = plt.figure()
    sb.boxplot(x=train_data_copy["age_loan"], y=train_data_copy["average salary"])
    plt.xlabel('age_loan', y=1.05, fontsize=15, labelpad=15)
    plt.ylabel('average salary', x=0.7, fontsize=15, labelpad=15)
    plt.title('Average salary per decade of age', fontdict={'fontsize': 14, 'fontweight': 'bold'})
    fig.set_size_inches(15, 10)
    plt.show()

In [ ]:
if show_graphs:
    train_data_copy = train_data.copy()
    train_data_copy["genre"] = train_data_copy["genre"].astype(str)
    train_data_copy["genre"].replace({"0": "Female","1": "Male"}, inplace=True)

    plt.figure(figsize=(15,10))
    plt.ylim(9000, 10000)
    sb.barplot(x = train_data_copy["genre"], y = train_data_copy["average salary"])
    plt.title("Average Salary by genre", fontdict={'fontsize': 14, 'fontweight': 'bold'})
    # display
    plt.show()

[back](#index)
#### Matrix <a id="matrix"></a>

train_data_w /test_data_w : train/test data where withdrawals in cash are replaced by withdrawals

In [ ]:
train_data.info()

In [ ]:
# Dropping ids
#train_data_no_ids = train_data.drop(['loan_id', 'account_id', 'district_id', 'disp_id', 'client_id', 'card_id', 'trans_id'], axis=1)
#train_data_no_ids = train_data.drop(['client_id', 'account_id', 'district_id', 'disp_id', 'card_id', 'trans_id', 'age_loan', 'age_account'], axis=1)
#test_data_no_ids = test_data.drop(['client_id', 'account_id', 'district_id', 'disp_id', 'card_id', 'trans_id', 'age_loan', 'age_account'], axis=1)
train_data_no_ids = train_data.drop(['account_id', 'district_id','age_loan_range_10-19','age_loan_range_20-29','age_loan_range_30-39','age_loan_range_40-49','age_loan_range_50-59','age_loan_range_60-69','age_account_range_10-19','age_account_range_20-29','age_account_range_30-39','age_account_range_40-49','age_account_range_50-59','age_account_range_60-69'], axis=1)
test_data_no_ids = test_data.drop(['account_id', 'district_id'], axis=1)


# Create correlation matrix
corr_matrix = train_data_no_ids.corr().abs()
plt.figure(figsize = (20,6))
sb.heatmap(corr_matrix, annot=True)

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
train_data_no_ids.drop(to_drop, axis=1, inplace=True)
test_data_no_ids.drop(to_drop, axis=1, inplace=True)

print("{} Dropped columns: {}".format(len(to_drop), to_drop) )

In [ ]:
train_data_no_ids.head()

In [ ]:
# Removing rows with nan values for status
#train_data_no_ids.dropna(subset=["status"], inplace=True)

#print(no_ids.drop_duplicates(inplace=True)
print(train_data_no_ids["status"].value_counts())

In [ ]:
train_data_no_ids.info()

In [ ]:
train_data_no_ids.head()

In [ ]:
all_inputs = train_data_no_ids[train_data_no_ids.columns.drop(['loan_id', 'status'])]
all_labels = train_data_no_ids['status'].values

# competition_inputs = test_data_no_ids.drop(columns=["loan_id"])
# all_ids_comp = test_data_no_ids['loan_id'].values

#all_inputs = train_data_no_ids.iloc[:, :-1].values
#all_labels = train_data_no_ids.iloc[:, -1].values

In [ ]:
all_inputs.isnull().sum().plot(kind='bar', figsize=(18,8), fontsize=14,);

all_inputs = all_inputs.drop(columns=["operation_cov_credit_cash","operation_cov_with_cash"])
print(all_inputs.isnull().sum())

all_inputs['amount_trans_std'].fillna(value=all_inputs['amount_trans_std'].mean(), inplace=True)
all_inputs['balance_std'].fillna(value=all_inputs['balance_std'].mean(), inplace=True)
all_inputs['type_cov_withdrawal'].fillna(value=all_inputs['type_cov_withdrawal'].mean(), inplace=True)

#all_inputs["operation_cov_credit_cash"].fillna(0, inplace=True)
#all_inputs["operation_cov_with_cash"].fillna(0, inplace=True)
#all_inputs["amount_trans_std"].fillna(0, inplace=True)
#all_inputs["balance_std"].fillna(0, inplace=True)
#all_inputs["type_cov_withdrawal"].fillna(0, inplace=True)

In [ ]:
all_inputs.isnull().sum()

In [ ]:
# get a test dataset with 25% of the credit_data_subset
(X_train, X_test, y_train, y_test) = train_test_split(all_inputs, all_labels, test_size=0.25, random_state=1)

sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
test_data_no_ids.info()

#### [back](#index)
## Classifiers <a class="anchor" id="classifiers"></a>

In [ ]:
model = 5

if model == 0:
    classifier = DecisionTreeClassifier()
elif model == 1:
    classifier = KNeighborsClassifier()
elif model == 2:
    classifier = SVC(probability=True)
elif model == 3:
    scaler = StandardScaler()

    # Fit only to the training data
    scaler.fit(X_train)

    # Now apply the transformations to the data:
    X_train_nn = scaler.transform(X_train)
    X_test_nn = scaler.transform(X_test)

    # Create the classifier
    classifier = MLPClassifier(random_state=1, max_iter=500) 
elif model == 4:
    classifier = LogisticRegression()
elif model == 5:
    classifier = GaussianNB()
elif model == 6:
    classifier = RandomForestClassifier(300)
elif model == 7:
    classifier = XGBClassifier()
elif model == 8:
    classifier = MLPClassifier(alpha=1, max_iter=1000)
elif model == 9:
    classifier = AdaBoostClassifier()
elif model == 10:
    classifier = VotingClassifier(
     estimators=[('dt', DecisionTreeClassifier()), ('svm', LinearSVC()), ('xgb', XGBClassifier())],
     voting='hard', weights=[1,1,1]
)
    
classifier.fit(X_train, y_train)
classifier_prediction = classifier.predict(X_test)

classifier_classification_report = classification_report(y_test, classifier_prediction, output_dict=True)

print(f"Classification report:\n{classification_report(y_test, classifier_prediction)}\n")

sb.set(font_scale=1.0)
 
ax = plt.subplot()

confusion_matrix_dtc = confusion_matrix(y_test, classifier_prediction)

sb.heatmap(confusion_matrix_dtc, annot=True, ax=ax, fmt="g")

ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Decision Tree <a class="anchor" id="decision-tree"></a>



In [ ]:
# Create the classifier
decision_tree_classifier = DecisionTreeClassifier()

# Feature Selection
decision_tree_classifier = RFECV(decision_tree_classifier, scoring='roc_auc')

# Train the classifier on the training set
decision_tree_classifier.fit(X_train, y_train)

dtc_prediction = decision_tree_classifier.predict(X_test)

dtc_classification_report = classification_report(y_test, dtc_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = decision_tree_classifier.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_dt = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_dt, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = decision_tree_classifier.predict(X_test) 
f1_dt = f1_score(y_test, predictions_test)
roc_dt = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_dt))
print(f"ROC: {roc_dt}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_dt = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_dt, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()


### Parameter Tunning <a class="anchor" id="parameter-tunning"></a>

In [ ]:
parameter_grid = {'criterion': ['gini', 'entropy'],
                  'splitter': ['best', 'random'],
                  'max_depth': range(1, 10),
                  'max_features': range(10,20)}

# grid_search = GridSearchCV(DecisionTreeClassifier(),
#                            param_grid=parameter_grid,
#                            cv=10,
#                            verbose=4,
#                            n_jobs=-1)

dt_classifier = DecisionTreeClassifier(min_samples_leaf = 10)

dt_grid_search = GridSearchCV(dt_classifier, scoring="precision_weighted", cv=10, param_grid=parameter_grid)
dt_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(dt_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = dt_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_dt = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_dt, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = dt_grid_search.predict(X_test) 
f1_dt_tun = f1_score(y_test, predictions_test)
roc_f1_dt = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_dt_tun))
print(f"ROC: {roc_f1_dt}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_dt = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_dt, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## K-Nearest Neighbor <a class="anchor" id="k-nearest-neighbor"></a>

In [ ]:
knn = KNeighborsClassifier()

# Feature Selection
# knn = RFECV(knn, scoring='roc_auc')

knn.fit(X_train, y_train)
knn_prediction = knn.predict(X_test)

knn_classification_report = classification_report(y_test, knn_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = knn.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_knn = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_knn, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = knn.predict(X_test) 
f1_knn = f1_score(y_test, predictions_test)
roc_knn = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_knn))
print(f"ROC: {roc_knn}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_knn = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_knn, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-2"></a>

In [ ]:
# parameter_grid = {'n_neighbors': [5,10,15,20],
#                   'weights': ['uniform', 'distance'],
#                   'algorithm': ['ball_tree', 'kd_tree', 'brute']}

# grid_search = GridSearchCV(KNeighborsClassifier(),
#                            param_grid=parameter_grid,
#                            scoring='precision_weighted',
#                            cv=10,
#                            n_jobs=3,
#                            verbose=4)

knn_classifier = KNeighborsClassifier()

parameter_grid = {'n_neighbors': [3,5,11,19],
                'weights': ['uniform','distance'],
                'p':[1,2],
                 'algorithm':['ball_tree', 'kd_tree', 'brute']}

knn_grid_search = GridSearchCV(knn_classifier, parameter_grid, scoring="roc_auc", n_jobs=-1, cv=10)
knn_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(knn_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = knn_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_knn = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_knn, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = knn_grid_search.predict(X_test) 
f1_knn_tun = f1_score(y_test, predictions_test)
roc_knn_tun = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_knn_tun))
print(f"ROC: {roc_knn_tun}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_knn = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_knn, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Support-Vector Machines <a class="anchor" id="support-vector-machines"></a>

In [ ]:
svc = SVC(probability=True)

# Feature Selection
# svc = RFECV(svc, scoring='roc_auc')

svc.fit(X_train, y_train)
svc_prediction = svc.predict(X_test)

svm_classification_report = classification_report(y_test, svc_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = svc.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_svc = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_svc, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = svc.predict(X_test) 
f1_svm = f1_score(y_test, predictions_test)
roc_svm = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_svm))
print(f"ROC: {roc_svm}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_svc = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_svc, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-3"></a>

In [ ]:
svm_classifier = SVC(probability=True)

parameter_grid = {'kernel': ['linear','rbf','poly']}

svm_grid_search = GridSearchCV(svm_classifier, scoring="roc_auc", cv=5, param_grid=parameter_grid)
svm_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(svm_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = svm_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_svm = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_svm, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = svm_grid_search.predict(X_test) 
print("F1 Score: {}".format(f1_score(y_test, predictions_test)))
print(f"ROC: {roc_auc_score(y_test, predictions_test)}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_svm = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_svm, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Neural Networks <a class="anchor" id="neural-networks"></a>

In [ ]:
scaler = StandardScaler()

# Feature Selection
#scaler = RFECV(scaler, scoring='roc_auc')

# Fit only to the training data
scaler.fit(X_train)

# Now apply the transformations to the data:
X_train_nn = scaler.transform(X_train)
X_test_nn = scaler.transform(X_test)

# Create the classifier
ANNClassifier = MLPClassifier(random_state=1, max_iter=500)

# Train the classifier on the training set
ANNClassifier.fit(X_train_nn, y_train)

predictions_test = ANNClassifier.predict(X_test_nn)

f1_nn = f1_score(y_test, predictions_test)
roc_nn = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_nn))
print(f"ROC: {roc_nn}")

confusion_matrix_ann = confusion_matrix(y_test,predictions_test)

nn_classification_report = classification_report(y_test, predictions_test, output_dict=True)
print(classification_report(y_test,predictions_test))

sb.set(font_scale=1.0)

ax = plt.subplot()

sb.heatmap(confusion_matrix_ann, annot=True, ax=ax, fmt="g")

ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

best_nn_classification_report = nn_classification_report

### Parameter Tunning <a class="anchor" id="parameter-tunning-4"></a>

In [ ]:
parameter_grid = {'activation': ['tanh','identity','logistic','relu'],
                  'solver': ['adam','lbfgs','sgd'],
                  'hidden_layer_sizes': [3,5,8,13,21,34],
                  'verbose': [True]}

cross_validation = StratifiedKFold(n_splits=10, shuffle=True)

grid_search = GridSearchCV(ANNClassifier,
                           param_grid=parameter_grid,
                           cv=cross_validation)

grid_search.fit(X_train, y_train)

print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))
print('Best estimator: {}'.format(grid_search.best_estimator_))

In [ ]:
ANNClassifier = grid_search.best_estimator_
yk_pred = ANNClassifier.predict(X_test)

best_nn_classification_report = classification_report(y_test, yk_pred, output_dict=True)

print("--- Improved model ---\n")
print(f"Classification report:\n{best_nn_classification_report(y_test, yk_pred)}\n")

sb.set(font_scale=1.0)

ax = plt.subplot()

confusion_matrix_ann = confusion_matrix(y_test, yk_pred)

sb.heatmap(confusion_matrix_ann, annot=True, ax=ax, fmt="g")

ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Logistic Regression <a class="anchor" id="logistic-regression"></a>

In [ ]:
lr = LogisticRegression()

# Feature Selection
lr = RFECV(lr, scoring='roc_auc')

lr.fit(X_train, y_train)
lr_prediction = lr.predict(X_test)

lr_classification_report = classification_report(y_test, lr_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = lr.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_lr = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_lr, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = lr.predict(X_test) 
f1_lr = f1_score(y_test, predictions_test)
roc_lr = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_lr))
print(f"ROC: {roc_lr}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_lr = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_lr, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-5"></a>

In [ ]:
lr_classifier = LogisticRegression()

lr_grid_search = GridSearchCV(lr_classifier, scoring="roc_auc", cv=5, param_grid={})
lr_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(lr_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = lr_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_lr = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_lr, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = lr_grid_search.predict(X_test) 
print("F1 Score: {}".format(f1_score(y_test, predictions_test)))
print(f"ROC: {roc_auc_score(y_test, predictions_test)}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_lr = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_lr, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Naive Bayes <a class="anchor" id="naive-bayes"></a>

In [ ]:
nb = GaussianNB()

# Feature Selection
# nb = RFECV(nb, scoring='roc_auc')

nb.fit(X_train, y_train)
nb_prediction = nb.predict(X_test)

nb_classification_report = classification_report(y_test, nb_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = nb.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_nb = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_nb, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = nb.predict(X_test) 
f1_nb = f1_score(y_test, predictions_test)
roc_nb = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_nb))
print(f"ROC: {roc_nb}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_nb = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_nb, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()


### Parameter Tunning <a class="anchor" id="parameter-tunning-6"></a>

In [ ]:
nb_classifier = GaussianNB()

nb_grid_search = GridSearchCV(nb_classifier, scoring="roc_auc", cv=5, param_grid={})
nb_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(nb_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = nb_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_nb = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_nb, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = nb_grid_search.predict(X_test) 
print("F1 Score: {}".format(f1_score(y_test, predictions_test)))
print(f"ROC: {roc_auc_score(y_test, predictions_test)}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_nb = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_nb, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Random Forest <a class="anchor" id="random-forest"></a>

In [ ]:
rf = RandomForestClassifier()

# Feature Selection
rf = RFECV(rf, scoring='roc_auc')

rf.fit(X_train, y_train)
rf_prediction = rf.predict(X_test)

rf_classification_report = classification_report(y_test, rf_prediction, output_dict=True)


print(53 * '=')
print("TRAIN")
predictions_train = rf.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_rf = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_rf, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = rf.predict(X_test) 
f1_rf = f1_score(y_test, predictions_test)
roc_rf = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_rf))
print(f"ROC: {roc_rf}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_rf = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_rf, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-7"></a>

In [ ]:
rf_classifier = RandomForestClassifier()

rf_grid_search = GridSearchCV(rf_classifier, scoring="roc_auc", cv=5, param_grid={})
rf_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(rf_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = rf_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_rf = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_rf, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = rf_grid_search.predict(X_test) 
print("F1 Score: {}".format(f1_score(y_test, predictions_test)))
print(f"ROC: {roc_auc_score(y_test, predictions_test)}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_rf = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_rf, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## XGBoost <a class="anchor" id="xgboost"></a>

In [ ]:
# xgb = XGBClassifier()

# Feature Selection
# xgb = RFECV(xgb, scoring='roc_auc')

# xgb.fit(X_train, y_train)
# xgb_prediction = xgb.predict(X_test)

# xgb_classification_report = classification_report(y_test, xgb_prediction, output_dict=True)

# print(f"Classification report:\n{classification_report(y_test, xgb_prediction, labels=np.unique(y_train))}\n")

# sb.set(font_scale=1.0)

# ax = plt.subplot()

# confusion_matrix_xgb = confusion_matrix(y_test, xgb_prediction)

# sb.heatmap(confusion_matrix_xgb, annot=True, ax=ax, fmt="g")

# ax.set_xlabel('Predicted Grades');
# ax.set_ylabel('Observed Grades');
# ax.set_title('Confusion Matrix');
# plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-8"></a>

#### [back](#index)
## MLP <a class="anchor" id="mlp"></a>

In [ ]:
mlp = MLPClassifier(alpha=1, max_iter=1000)

# Feature Selection
# mlp = RFECV(mlp, scoring='roc_auc')

mlp.fit(X_train, y_train)
mlp_prediction = mlp.predict(X_test)

mlp_classification_report = classification_report(y_test, mlp_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = mlp.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_mlp = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_mlp, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = mlp.predict(X_test) 
f1_mlp = f1_score(y_test, predictions_test)
roc_mlp = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_mlp))
print(f"ROC: {roc_mlp}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_mlp = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_mlp, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-9"></a>

In [ ]:
mlp_classifier = MLPClassifier(random_state=1, early_stopping=False)

mlp_grid_search = GridSearchCV(mlp_classifier, scoring="roc_auc", cv=10, param_grid={})
mlp_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(mlp_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = mlp_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_mlp = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_mlp, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = mlp_grid_search.predict(X_test) 
print("F1 Score: {}".format(f1_score(y_test, predictions_test)))
print(f"ROC: {roc_auc_score(y_test, predictions_test)}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_mlp = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_mlp, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Ada Boost <a class="anchor" id="ada-boost"></a>

In [ ]:
boost = AdaBoostClassifier()

# Feature Selection
boost = RFECV(boost, scoring='roc_auc')

boost.fit(X_train, y_train)
boost_prediction = boost.predict(X_test)

boost_classification_report = classification_report(y_test, boost_prediction, output_dict=True)

print(53 * '=')
print("TRAIN")
predictions_train = boost.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
print()
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_boost = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_boost, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = boost.predict(X_test) 
f1_boost = f1_score(y_test, predictions_test)
roc_boost = roc_auc_score(y_test, predictions_test)
print("F1 Score: {}".format(f1_boost))
print(f"ROC: {roc_boost}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_boost = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_boost, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

### Parameter Tunning <a class="anchor" id="parameter-tunning-10"></a>

In [ ]:
ada_classifier = AdaBoostClassifier(random_state=0)

ada_grid_search = GridSearchCV(ada_classifier, scoring="roc_auc", cv=3, param_grid={})
ada_grid_search.fit(X_train, y_train)
print('Best score: {}'.format(ada_grid_search.best_score_))


print(53 * '=')
print("TRAIN")
predictions_train = ada_grid_search.predict(X_train)
print("F1 Score: {}".format(f1_score(y_train, predictions_train)))
print(f"ROC: {roc_auc_score(y_train, predictions_train)}")
print("Classification Report: ")
print(classification_report(y_train, predictions_train, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_ada = confusion_matrix(y_train, predictions_train)
sb.heatmap(confusion_matrix_ada, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

print(53 * '=')
print("TEST")
predictions_test = ada_grid_search.predict(X_test) 
print("F1 Score: {}".format(f1_score(y_test, predictions_test)))
print(f"ROC: {roc_auc_score(y_test, predictions_test)}")
print("Classification Report: ")
print(classification_report(y_test, predictions_test, target_names=['not pay', 'pay']))
sb.set(font_scale=1.0)
ax = plt.subplot()
confusion_matrix_ada = confusion_matrix(y_test, predictions_test)
sb.heatmap(confusion_matrix_ada, annot=True, ax=ax, fmt="g")
ax.set_xlabel('Predicted Grades');
ax.set_ylabel('Observed Grades');
ax.set_title('Confusion Matrix');
plt.show()

#### [back](#index)
## Voting <a class="anchor" id="voting"></a>

In [ ]:
# vote = VotingClassifier(
#      estimators=[('dt', DecisionTreeClassifier()), ('svm', LinearSVC()), ('xgb', XGBClassifier())],
#      voting='hard', weights=[1,1,1]
# )

# Feature Selection
# classifier = RFECV(vote, scoring='roc_auc')

# vote.fit(X_train, y_train)
# vote_prediction = vote.predict(X_test)

# vote_classification_report = classification_report(y_test, vote_prediction, output_dict=True)

# print(f"Classification report:\n{classification_report(y_test, vote_prediction, labels=np.unique(y_train))}\n")

# sb.set(font_scale=1.0)

# ax = plt.subplot()

# confusion_matrix_vote = confusion_matrix(y_test, vote_prediction)

# sb.heatmap(confusion_matrix_vote, annot=True, ax=ax, fmt="g")

# ax.set_xlabel('Predicted Grades');
# ax.set_ylabel('Observed Grades');
# ax.set_title('Confusion Matrix');
# plt.show()

In [ ]:
#table = go.Figure(data=[go.Table(header=dict(values=['Algorithm', 'Accuracy Normal']),
#                 cells=dict(values=[['DT', 'KNN', 'SVM', 'NN', 'LR', 'NB', 'RF', 'XGB', 'MLP', 'BOOST', 'VOTE'], [dtc_classification_report['accuracy'], knn_classification_report['accuracy'], svm_classification_report['accuracy'], nn_classification_report['accuracy'], lr_classification_report['accuracy'], nb_classification_report['accuracy'], rf_classification_report['accuracy'], xgb_classification_report['accuracy'], mlp_classification_report['accuracy'], boost_classification_report['accuracy'], vote_classification_report['accuracy']]]))
#                     ])

table = go.Figure(data=[go.Table(header=dict(values=['Algorithm', 'Accuracy Normal', 'F1 Score Normal', 'ROC Score Normal']),
                 cells=dict(values=[['DT', 'KNN', 'SVM', 'NN', 'LR', 'NB', 'RF', 'MLP', 'BOOST'], 
                                    [dtc_classification_report['accuracy'], knn_classification_report['accuracy'], svm_classification_report['accuracy'], nn_classification_report['accuracy'], lr_classification_report['accuracy'], nb_classification_report['accuracy'], rf_classification_report['accuracy'], mlp_classification_report['accuracy'], boost_classification_report['accuracy']], 
                                   [f1_dt, f1_knn, f1_svm, f1_nn, f1_lr, f1_nb, f1_rf, f1_mlp, f1_boost], [roc_dt, roc_knn, roc_svm, roc_nn, roc_lr, roc_nb, roc_rf, roc_mlp, roc_boost]]))
                     ])

table.show()

In [ ]:
test_data_no_ids.info()

#### [back](#index)
## Apply Model <a class="anchor" id="apply-model"></a>

In [ ]:
test_inputs = test_data_no_ids.drop(columns=['status', 'loan_id'])

pred_comp = boost.predict_proba(test_inputs)

pred_comp = pd.DataFrame(pred_comp, columns=['col2', 'Predicted'])

pred_comp.drop('col2', axis=1, inplace=True)
all_ids_comp = pd.DataFrame(all_ids_comp, columns=['Id'])
# pred_comp = [(1-pred_comp[n]) for n in range(0, len(pred_comp))]
# print(pred_comp)
results = pd.concat([all_ids_comp, pred_comp], axis=1)
results = results.rename(columns={"loan_id":"Id"})
results.to_csv('results.csv', index = False)

print(results)